In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs_min = "20260107"
day_obs_max = "20260203"
not_times_square = True

# Visit acquisition rate and delivered image quality from {{ params.day_obs_min }} to {{ params.day_obs_max }} # 

Evaluate simply number of science visits and delivered image quality compared to average baseline v5.1 numbers.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

from astropy.time import Time, TimeDelta
import astropy.units as u

import logging
import warnings

from rubin_scheduler.site_models import SeeingModel

from rubin_nights import connections
from rubin_nights.augment_visits import augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.dayobs_utils as rn_dayobs

try:
    from rubin_nights.reference_values import (
        SCIENCE_PROGRAMS,
        GAUSSIAN_FWHM_OVER_SIGMA,
        PLATESCALE,
    )
except ImportError:
    print("Specifying science programs from notebook values")
    SCIENCE_PROGRAMS = (
        "BLOCK-407",
        "BLOCK-408",
        "BLOCK-416",
        "BLOCK-417",
        "BLOCK-419",
        "BLOCK-421",
    )
    GAUSSIAN_FWHM_OVER_SIGMA = 2.0 * np.sqrt(2.0 * np.log(2.0))
    PLATESCALE = 0.2

try:
    not_times_square
except NameError:
    not_times_square = False

In [ ]:
# Turn day_obs limits into astropy times

t_start = Time(
    f"{rn_dayobs.day_obs_int_to_str(day_obs_min)}T12:00:00", format="isot", scale="utc"
)
t_end = Time(
    f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format="isot", scale="utc"
) + TimeDelta(1, format="jd")

one_day = TimeDelta(1, format="jd")
days = t_start + one_day * np.arange(0, (t_end - t_start).jd)
day_obs_list = [
    rn_dayobs.day_obs_str_to_int(rn_dayobs.time_to_day_obs(d)) for d in days
]

In [ ]:
# First - query consdb for visit information

endpoints = connections.get_clients()

# refresh_visits and not_times_square just make it easier to rerun the notebook yourself without re-downloading info
refresh_visits = True
if refresh_visits:
    visits = endpoints["consdb_tap"].get_visits(
        "lsstcam",
        t_start,
        t_end,
        visit_constraint=f"science_program in {SCIENCE_PROGRAMS}",
    )
    if not_times_square:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            visits.to_hdf("v_now.h5", key="visits")
else:
    visits = pd.read_hdf("v_now.h5")

print(
    f"Retrieved {len(visits)} science visits from consdb between {t_start.iso} and {t_end.iso}"
)

ave_n_vis_per_night = (
    visits.groupby("day_obs").agg({"visit_id": "count"}).visit_id.mean()
)
print(f"Average number of visits per night: {ave_n_vis_per_night:.2f}")

In [ ]:
try:
    from rubin_nights.dayobs_utils import estimated_baseline_visit_range
except ImportError:
    # This isn't in a tagged version of rubin_nights yet, and will only be in RSP week after tag
    def estimated_baseline_visit_range(
        day_obs: int, relative_performance=1.0
    ) -> dict[str, int]:
        """Estimate an average and likely upper limit for the number of visits on
        a given day_obs.

        Parameters
        ----------
        day_obs
            The day of interest.
        relative_performance
            The expected open shutter fraction ratio compared to the baseline
            value used in the simulations.
            The simulations used to estimate open shutter fraction here are v5.1.

        Returns
        -------
        estimate_visits : `dict` [`str` : `int`]
            A dictionary with `n_vis_ave` and `n_vis_high` keys, containing
            a range of estimated nvisits values. These estimates are based on
            open shutter fraction and night length.
        """
        sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)
        night_length = (sunrise - sunset).jd * 24 * 60 * 60  # seconds
        night = rn_dayobs.day_obs_to_time(day_obs)
        night_min = Time("2025-12-21T12:00:00")
        # The mean open shutter fraction in v5.1 runs is ~0.74
        # but this doesn't match average nvisits without modulation
        open_shutter_fraction = 0.74 - 0.02 * np.cos(
            (night - night_min).jd / 365 * 2 * np.pi
        )
        estimate_nvis_ave = (
            open_shutter_fraction * night_length / 30
        )  # assume 30s visits
        estimate_nvis_ave = int(np.floor(estimate_nvis_ave * relative_performance))
        # The upper envelope of nvisits/night over many simulations + years
        # matches more closely with 0.78
        open_shutter_fraction = 0.78
        estimate_nvis_hi = (
            open_shutter_fraction * night_length / 30
        )  # assume 30s visits
        estimate_nvis_hi = int(np.floor(estimate_nvis_hi * relative_performance))
        return {"n_vis_ave": estimate_nvis_ave, "n_vis_high": estimate_nvis_hi}


# Add estimate of expected visits, from ensemble of v5.1 simulations and a fit
dd = []
for day_obs in day_obs_list:
    dd.append(estimated_baseline_visit_range(day_obs))
nvis_expected = pd.DataFrame(dd, index=day_obs_list)
print(
    f"Total number of estimated expected visits would be {nvis_expected.n_vis_ave.sum()} between {t_start.iso} and {t_end.iso}"
)
print(f"Average number estimated per night {nvis_expected.n_vis_ave.mean():.2f}")

In [ ]:
# Join acquired number of visits per day with expected.
acq_per_day = (
    visits.groupby("day_obs")
    .agg({"visit_id": "count"})
    .rename({"visit_id": "n_vis_acq"}, axis=1)
)
nvis_per_day = nvis_expected.join(acq_per_day, how="outer").fillna(0)
nvis_per_day["ratio"] = nvis_per_day["n_vis_acq"] / nvis_per_day["n_vis_ave"]
mean_nvis_acq_expected = nvis_per_day.ratio.mean()
print(
    f"Comparing acquired science visits per night to the estimated expected number of visits per night - mean ratio is {mean_nvis_acq_expected:.2f}"
)
print(
    f"Comparing the total bulk number of visits (not per night) we've acquired {len(visits) / nvis_expected.n_vis_ave.sum():.2f} of what we might have."
)

In [ ]:
# What would the fE box approximately translate to?

print("Using baseline_v5.1.0_10yrs as the comparison")
fraction_wfd = 1692518.00 / 2075536.00
fO_comparison = 762.00 / 825 * 0.9 / fraction_wfd
print(f"fO comparison - median_nvis / 825 * 0.9 / fraction_wfd = {fO_comparison:.2f}")

# Assuming fA = 0.99
fA = 0.99
print(f"Assuming fA = {fA:.2f}")


# And fill in a speculative value for fS for now
def fS(atm, sys):
    fwhm = np.sqrt(atm**2 + sys**2)
    delta_m = 2.5 * np.log10(0.7 / fwhm)
    f_s = 10 ** (0.8 * delta_m)
    return f_s


fS = 1.3 * (fS(0.6, 0.5) / fS(0.6, 0.4))
print(f"And approximating fS as {fS:.2f}")

fE = fO_comparison * fA * fS
print(f"Means these combine to fE {fE:.2f}")
# We could just assume  (n_vis_acquired / n_vis_expected) should multiply fE above
# Then minimum fE = 0.7 -> n_vis_acquired / n_vis_expected >= 0.62
min_nvis_ratio = 0.62
print(
    f"So looking for min ratio of n_vis_acquired / n_vis_expected of {min_nvis_ratio:.2f}"
)

In [ ]:
# what would system contribution to image quality box translate to?
seeing_model = SeeingModel(telescope_seeing=0.4)
print(
    f"seeing model assuming system contribution at zenith {seeing_model.fwhm_system_zenith:.2f}"
)
fwhm_mean = visits.psf_sigma_median.mean() * PLATESCALE * GAUSSIAN_FWHM_OVER_SIGMA
airmass_mean = visits.airmass.mean()
print(f"using airmass mean value of {airmass_mean:.2f}")
min_diq = seeing_model(0.7, airmass=airmass_mean)["fwhmEff"][2]
print(
    f"Translate fiducial atmospheric contribution of 0.7 arcsec to diq of {min_diq:.2f}"
)

print(f"Mean DIMM1 value (when available): {visits.dimm_seeing.mean():.2f}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8))
ax.plot(mean_nvis_acq_expected, fwhm_mean, "*", markersize=12)


x = np.arange(min_nvis_ratio, 1.5, 0.01)
y1 = 0.6
y2 = min_diq
ax.fill_between(x, y1=y1, y2=y2, color="green", alpha=0.3)

ax.set_xlim(0, 1.4)
ax.set_xlabel("Number of visits / expected number - per night", fontsize="x-large")
ax.set_ylim(0.7, 1.3)
ax.set_ylabel("Mean of FWHM median per visit (arcseconds)", fontsize="x-large")
ax.grid(alpha=0.3)

_ = ax.set_title(f"Average {day_obs_min} to {day_obs_max}", fontsize="x-large")

In [ ]:
q = visits.groupby("day_obs").agg({"psf_sigma_median": "mean"})
q["fwhm_mean"] = q.psf_sigma_median * PLATESCALE * GAUSSIAN_FWHM_OVER_SIGMA
q = nvis_per_day.join(q)

plt.figure(figsize=(15, 6))
x = np.arange(0, len(q))
plt.plot(x, q.ratio, marker=".", label="nvisits acquired/expected")
plt.plot(x, q.fwhm_mean, marker=".", label="mean fwhm median per visit")
plt.legend()
plt.ylim(0, 2)
plt.grid(alpha=0.3)
_ = plt.xticks(x, labels=q.index, rotation=90)